In [68]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
import re
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [69]:
import xml.etree.ElementTree as etree
source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"

In [70]:
matlab_file = 'goodwin-minsky/goodwin-minsky-with-names.m'
template_xmile = 'template-xmile.xml'

# Parseo archivo Matlab

In [71]:
with open(matlab_file, 'rb') as f:
    matlab_code = f.read()
    print(matlab_code)

function f=f(x,t)
Capital=x(1);
Debt=x(2);
LaborProductivity=x(3);
Population=x(4);
wageRate=x(5);
piS=10;
employmentRateZero=0.9;
employmentRateStable=10;
deltaKReal=0.1;
piZ=0.03;
rateInterestOnLoans=0.04;
velocityOfMoney=2;
Betaa=0.015;
Alphaa=0.025;
Interest=(rateInterestOnLoans)*(Debt);
Output=(Capital)/((velocityOfMoney));
Labor=(Output)/((LaborProductivity));
Wages=(wageRate)*(Labor);
employmentRateValue=(Labor)/((Population));
wageFunction=(employmentRateStable)*(employmentRateValue-(employmentRateZero));
ProfitGrossReal=Output-(Wages);
omega=(Wages)/((Output));
ProfitNet=(ProfitGrossReal)-(Interest);
piR=(ProfitNet)/((Capital));
InvestmentFunctionReal=(piR-(piZ))*(piS);
InvestmentGross=(InvestmentFunctionReal)*(Output);
InvestmentNetReal=InvestmentGross-((Capital)*(deltaKReal));
f(1)=InvestmentNetReal;
f(2)=InvestmentGross;
f(3)=(Alphaa)*(LaborProductivity);
f(4)=(Population)*(Betaa);
f(5)=(wageFunction)*(wageRate);
endfunction;

x0(1)=300;
x0(2)=0;
x0(3)=1;
x0(4)=120;
x0(5)=0

In [40]:
from py_expression_eval import Parser
parser = Parser()

In [14]:
# stocks ids
stocks_ids_re = re.compile(r"[\w]+=x\(\d+\)")
stocks_ids = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
              re.search(re.compile(r"x\((.+)\)"), exp).group(1)
              for exp in stocks_ids_re.findall(matlab_code)}
print(stocks_ids)

{'LaborProductivity': '3', 'wageRate': '5', 'Debt': '2', 'Population': '4', 'Capital': '1'}


In [15]:
stocks_init_values_re = re.compile(r"x0\(\d+\)=\d+[\.\d+]*")
stocks_init_values_dict = {re.search(re.compile(r"x0\((.+)\)"), exp).group(1) : 
                    re.search(re.compile(r"=(.+)"), exp).group(0)[1:]
                    for exp in stocks_init_values_re.findall(matlab_code)}
#print(stocks_init_values_dict)

# Joineo los 'stocks' con sus valores iniciales Capital=x(1) => x0(1)=300
stocks_init_values_re = re.compile(r"[\w]+=x\(\d+\)")
stocks_init_values = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
                    stocks_init_values_dict[re.search(re.compile(r"x\((.+)\)"), exp).group(1)]
                    for exp in stocks_init_values_re.findall(matlab_code)}
print(stocks_init_values)

{'LaborProductivity': '1', 'wageRate': '0.8', 'Debt': '0', 'Population': '120', 'Capital': '300'}


In [43]:
stocks_change_re = re.compile(r"f\(\d+\)=[\w\(\)\+\-\*\/\.]+")
stocks_change_dict = {re.search(re.compile(r"f\((\d+)\)"), exp).group(1) : 
                      parser.parse(re.search(re.compile(r"=(.+)"), exp).group(0)[1:])
                      for exp in stocks_change_re.findall(matlab_code)}
#print(stocks_change_dict)

# Joineo los 'stocks' con sus rates de cambio Capital=x(1) => f(1)=InvestmentNetReal
stocks_change = {k : stocks_change_dict[stocks_ids[k]]
                 for k,v in stocks_init_values.iteritems()}
print({k : v.toString() for k,v in stocks_change.iteritems()})

{'LaborProductivity': '(Alphaa*LaborProductivity)', 'wageRate': '(wageFunction*wageRate)', 'Debt': 'InvestmentGross', 'Population': '(Population*Betaa)', 'Capital': 'InvestmentNetReal'}


In [44]:
auxs_init_values_re = re.compile(r"[\w]+=\d+[\.\d+]*")
auxs_init_values = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
                    re.search(re.compile(r"=(.+)"), exp).group(0)[1:] 
                    for exp in auxs_init_values_re.findall(matlab_code)}
print(auxs_init_values)

{'piS': '10', 'Betaa': '0.015', 'rateInterestOnLoans': '0.04', 'piZ': '0.03', 'Alphaa': '0.025', 'employmentRateZero': '0.9', 'employmentRateStable': '10', 'deltaKReal': '0.1', 'velocityOfMoney': '2'}


In [45]:
# TODO : solo acepta operaciones de +, -, *, /
auxs_equations_re = re.compile(r"[\w]+=[\w\(\)\+\-\*\/\.]+")
auxs_equations = {re.search(re.compile(r"(.+)="), exp).group(0)[:-1] : 
                    parser.parse(re.search(re.compile(r"=(.+)"), exp).group(0)[1:])
                    for exp in auxs_equations_re.findall(matlab_code)[1:]
                    if re.search(re.compile(r"(.+)="), exp).group(0)[:-1] not in
                    auxs_init_values.keys() + stocks_init_values.keys()}
print(auxs_equations.keys())
print([exp.simplify({}).toString() for exp in auxs_equations.values()])

['wageFunction', 'InvestmentGross', 'ProfitNet', 'employmentRateValue', 'InvestmentFunctionReal', 'Labor', 'ProfitGrossReal', 'Interest', 'Output', 'Wages', 'omega', 'piR', 'InvestmentNetReal']
['(employmentRateStable*(employmentRateValue-employmentRateZero))', '(InvestmentFunctionReal*Output)', '(ProfitGrossReal-Interest)', '(Labor/Population)', '((piR-piZ)*piS)', '(Output/LaborProductivity)', '(Output-Wages)', '(rateInterestOnLoans*Debt)', '(Capital/velocityOfMoney)', '(wageRate*Labor)', '(Wages/Output)', '(ProfitNet/Capital)', '(InvestmentGross-(Capital*deltaKReal))']


## Estructura de datos con informacion de stocks / flows / auxs

In [75]:
# stocks_init_values | auxs_init_values | flows_auxs_equation
stocks = [
    {'name' : k, 'eqn' : v, 'inflow' : 'chg'+k} 
    for k,v in stocks_init_values.iteritems()
]
flows = [
    {'name' : 'chg'+k, 'eqn' : stocks_change[k].toString()}
    for k,v in stocks_init_values.iteritems()
]
# Auxs : definidas de forma DIRECTA / INDIRECTA
auxs = [
    {'name' : k, 'eqn' : v.toString()}
    for k,v in auxs_equations.iteritems()
] + [
    {'name' : k, 'eqn' : v}
    for k,v in auxs_init_values.iteritems()
]
dependencies = [
    {'name' : 'chg'+k, 'inputs' : stocks_change[k].variables()}
    for k,v in stocks_init_values.iteritems()
] + [
    {'name' : k, 'inputs' : v.variables()}
    for k,v in auxs_equations.iteritems()
]


In [76]:
stocks

[{'eqn': '1', 'inflow': 'chgLaborProductivity', 'name': 'LaborProductivity'},
 {'eqn': '0.8', 'inflow': 'chgwageRate', 'name': 'wageRate'},
 {'eqn': '0', 'inflow': 'chgDebt', 'name': 'Debt'},
 {'eqn': '120', 'inflow': 'chgPopulation', 'name': 'Population'},
 {'eqn': '300', 'inflow': 'chgCapital', 'name': 'Capital'}]

In [77]:
flows

[{'eqn': '(Alphaa*LaborProductivity)', 'name': 'chgLaborProductivity'},
 {'eqn': '(wageFunction*wageRate)', 'name': 'chgwageRate'},
 {'eqn': 'InvestmentGross', 'name': 'chgDebt'},
 {'eqn': '(Population*Betaa)', 'name': 'chgPopulation'},
 {'eqn': 'InvestmentNetReal', 'name': 'chgCapital'}]

In [78]:
auxs

[{'eqn': '(employmentRateStable*(employmentRateValue-employmentRateZero))',
  'name': 'wageFunction'},
 {'eqn': '(InvestmentFunctionReal*Output)', 'name': 'InvestmentGross'},
 {'eqn': '(ProfitGrossReal-Interest)', 'name': 'ProfitNet'},
 {'eqn': '(Labor/Population)', 'name': 'employmentRateValue'},
 {'eqn': '((piR-piZ)*piS)', 'name': 'InvestmentFunctionReal'},
 {'eqn': '(Output/LaborProductivity)', 'name': 'Labor'},
 {'eqn': '(Output-Wages)', 'name': 'ProfitGrossReal'},
 {'eqn': '(rateInterestOnLoans*Debt)', 'name': 'Interest'},
 {'eqn': '(Capital/velocityOfMoney)', 'name': 'Output'},
 {'eqn': '(wageRate*Labor)', 'name': 'Wages'},
 {'eqn': '(Wages/Output)', 'name': 'omega'},
 {'eqn': '(ProfitNet/Capital)', 'name': 'piR'},
 {'eqn': '(InvestmentGross-(Capital*deltaKReal))',
  'name': 'InvestmentNetReal'},
 {'eqn': '10', 'name': 'piS'},
 {'eqn': '0.015', 'name': 'Betaa'},
 {'eqn': '0.04', 'name': 'rateInterestOnLoans'},
 {'eqn': '0.03', 'name': 'piZ'},
 {'eqn': '0.025', 'name': 'Alphaa'},


In [79]:
dependencies

[{'inputs': ['Alphaa', 'LaborProductivity'], 'name': 'chgLaborProductivity'},
 {'inputs': ['wageFunction', 'wageRate'], 'name': 'chgwageRate'},
 {'inputs': ['InvestmentGross'], 'name': 'chgDebt'},
 {'inputs': ['Population', 'Betaa'], 'name': 'chgPopulation'},
 {'inputs': ['InvestmentNetReal'], 'name': 'chgCapital'},
 {'inputs': ['employmentRateStable',
   'employmentRateValue',
   'employmentRateZero'],
  'name': 'wageFunction'},
 {'inputs': ['InvestmentFunctionReal', 'Output'], 'name': 'InvestmentGross'},
 {'inputs': ['ProfitGrossReal', 'Interest'], 'name': 'ProfitNet'},
 {'inputs': ['Labor', 'Population'], 'name': 'employmentRateValue'},
 {'inputs': ['piR', 'piZ', 'piS'], 'name': 'InvestmentFunctionReal'},
 {'inputs': ['Output', 'LaborProductivity'], 'name': 'Labor'},
 {'inputs': ['Output', 'Wages'], 'name': 'ProfitGrossReal'},
 {'inputs': ['rateInterestOnLoans', 'Debt'], 'name': 'Interest'},
 {'inputs': ['Capital', 'velocityOfMoney'], 'name': 'Output'},
 {'inputs': ['wageRate', 'Lab

# Generacion de archivo XMILE

In [80]:
model_name = 'test'
time_start = 1
time_stop = 2
time_delta = 0.5

In [81]:
context = {
    'model_name' : model_name,
    'time_start' : time_start,
    'time_stop'  : time_stop,
    'time_delta' : time_delta, 
    'stocks' : stocks,
    'flows'  : flows,
    'auxs'   : auxs,
    'dependencies' : dependencies
}

In [83]:
#print(render_template(template_xmile, context))

In [95]:
with open('goodwin-minsky/test-traduccion-matlab-xmile.xmile', 'w') as f:
    f.write(render_template(template_xmile, context))